# Tube rearrangement

Import the output file from the **tube extraction** stage:

In [2]:
from os import path, getcwd
from pcgvs.extraction import load_tubes_with_pandas

# Generating an OS safe path.
p = path.join(getcwd(), 'synopsis')
p = path.join(p, 'tubes')
p = path.join(p, 'exp')
p = path.join(p, 'tracks')
p = path.join(p, '/Users/nguyenduy/Desktop/pcgvs-main/notebooks/data/test2.txt')

dataframe = load_tubes_with_pandas(p)
dataframe.head(10)

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,frame,tag,x,y,w,h
0,169,1,40,242,25,21
1,170,1,39,240,27,24
2,171,1,39,239,27,25
3,174,1,37,238,30,26
4,175,1,38,236,25,27
5,176,1,36,234,29,30
6,176,4,108,244,23,20
7,177,1,35,232,33,31
8,177,4,109,244,24,20
9,178,1,34,231,34,33


Transform the dataframe to a list of Tube objects (see specifications at `pcgvs.extraction.Tube`) 

In [55]:
from pcgvs.extraction import load_tubes_from_pandas_dataframe

tubes = load_tubes_from_pandas_dataframe(dataframe)

for tube in tubes:
    print(f'tube of object {tube.tag} starts at frame {tube.sframe} and end at frame {tube.eframe}')
 

tube of object 1 starts at frame 169 and end at frame 367
tube of object 4 starts at frame 176 and end at frame 359
tube of object 6 starts at frame 384 and end at frame 473
tube of object 8 starts at frame 1076 and end at frame 1232
tube of object 10 starts at frame 1102 and end at frame 1154
tube of object 11 starts at frame 1115 and end at frame 1151
tube of object 14 starts at frame 1203 and end at frame 1229
tube of object 17 starts at frame 1314 and end at frame 1463
tube of object 19 starts at frame 1610 and end at frame 1751
tube of object 20 starts at frame 1797 and end at frame 1874
tube of object 21 starts at frame 1803 and end at frame 1966
tube of object 23 starts at frame 1818 and end at frame 1890
tube of object 24 starts at frame 1854 and end at frame 1867
tube of object 25 starts at frame 1857 and end at frame 1969
tube of object 26 starts at frame 2168 and end at frame 2265
tube of object 27 starts at frame 2426 and end at frame 2604
tube of object 28 starts at frame 

We use the tube rearrangement algorithm from [He Et al.](https://www.sciencedirect.com/science/article/pii/S0925231216313406) along with some of the optimizations proposed in [Allegra Et al.](https://ieeexplore.ieee.org/document/8803795).

The first step of the algorithm consist in computing the relations between the tubes. The relation between two generic tubes $T_1$ and $T_2$ can be one of the following:

- Irrilevant relation (`None`): the tubes trajectories doesn't intersect
- Intersecting relation (`INT`): the tubes trajectories intersect in some frames
- Overlapping relation (`OVL`): the tubes trajectories overlap in a long sequence of frames

In order to distinguish the intersection and the overlapping, the paper specify a threshold set to 5 frames, meaning that if the frames overlaps for more than 5 consecutive frames, then it's an overlapping relation, otherwise it's an intersecting relation.

In [56]:
from pcgvs.aggregation.relations import RelationsMap
relations = RelationsMap(tubes)
print(relations)

100%|██████████| 552/552 [00:17<00:00, 31.67it/s]

[1]:	(1)None	(4)OVL	(6)OVL	(8)OVL	(10)OVL	(11)OVL	(14)OVL	(17)OVL	(19)OVL	(20)OVL	(21)OVL	(23)OVL	(24)OVL	(25)OVL	(26)OVL	(27)OVL	(28)OVL	(30)OVL	(31)OVL	(32)OVL	(34)OVL	(35)OVL	(36)OVL	(38)OVL	
[4]:	(1)OVL	(4)None	(6)OVL	(8)OVL	(10)None	(11)None	(14)OVL	(17)OVL	(19)OVL	(20)None	(21)OVL	(23)None	(24)None	(25)OVL	(26)OVL	(27)OVL	(28)None	(30)None	(31)OVL	(32)OVL	(34)OVL	(35)OVL	(36)OVL	(38)OVL	
[6]:	(1)OVL	(4)OVL	(6)None	(8)OVL	(10)OVL	(11)OVL	(14)OVL	(17)OVL	(19)OVL	(20)OVL	(21)OVL	(23)OVL	(24)OVL	(25)OVL	(26)OVL	(27)OVL	(28)OVL	(30)OVL	(31)OVL	(32)OVL	(34)OVL	(35)OVL	(36)OVL	(38)OVL	
[8]:	(1)OVL	(4)OVL	(6)OVL	(8)None	(10)OVL	(11)OVL	(14)OVL	(17)OVL	(19)OVL	(20)OVL	(21)OVL	(23)OVL	(24)OVL	(25)OVL	(26)OVL	(27)OVL	(28)OVL	(30)OVL	(31)OVL	(32)OVL	(34)OVL	(35)OVL	(36)OVL	(38)OVL	
[10]:	(1)OVL	(4)None	(6)OVL	(8)OVL	(10)None	(11)OVL	(14)None	(17)OVL	(19)OVL	(20)None	(21)INT	(23)None	(24)None	(25)OVL	(26)None	(27)None	(28)None	(30)None	(31)OVL	(32)OVL	(34)INT	(35)INT	(36)None	(38)None	
[11]:	

In [57]:
f = open("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/data/overlap_frame.txt", "w")
f.write(str(relations))
f.close()

In [58]:
f = open("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/data/overlap_frame.txt", "r")
string = f.read()
list = string.split("\t")
f.close()

list_OVL=[]
list_count=[]
for i in list:
    list_OVL.append(i[4:])
for j in list_OVL:
    if j=='OVL' or j=='VL' or j=='INT':
        list_count.append('1')

num_object=len(dataframe.index)
overlap_count=len(list_count)/2
if overlap_count>(num_object/10):
    q=15
elif overlap_count>(num_object/13):
    q=8
elif overlap_count>(num_object/16):
    q=3

print(q)

8


Phần này a viết để tìm frame có object overlap. Sau khi overlap thì sẽ trả về frame rồi a viết ra file a.txt số frame của object đó. Sau đấy thì từ a.txt kiểm tra frame của object va chạm rồi  xử lí bằng addWeighted trong OpenCV để làm trong suốt 2 object va chạm với backgroud trong Overlap/colission.py . Cuối cùng trả về file object synopsis/patches để ghép object vào video trong notbook 4
 

In [59]:
# from tqdm import tqdm
# from itertools import permutations

In [60]:
# def compute(tubes,a):
#         n = len(tubes) 
#         for Ta, Tb in tqdm(permutations(tubes, 2), total=n*(n-1)):
#             if Ta == Tb: continue
#             ffintersec = None # First frame Overlap
#             lfintersec = None # Last frame Overlap
#             for adata in Ta:
#                 for bdata in Tb:
#                     frame = adata[4]
#                     if frames_intersect(adata, bdata):
#                         ffintersec = frame if ffintersec is None else a.append(ffintersec)
#                         lfintersec = frame
#                         a.append(lfintersec)
#         return a

# def frames_intersect(adata, bdata):
#         xa, ya, wa, ha, _ = adata
#         xb, yb, wb, hb, _ = bdata
#         l_ax, l_ay = xa, ya             # Top-left point of square A
#         r_ax, r_ay = xa + wa, ya + ha   # Bottom-right point of square A
#         l_bx, l_by = xb, yb             # Top-left point of square B
#         r_bx, r_by = xb + wb, yb + hb   # Bottom-right point of square B
#         # Check if one square has empty area.
#         if l_ax == r_ax or l_ay == r_ay or l_bx == r_bx or l_by == r_by:
#             return False
#         # Check if one square stands above the other.
#         if r_ay < l_by or r_by < l_ay:
#             return False
#         # Check if one square stands on the left of the other
#         if r_ax < l_bx or r_bx < l_ax:
#             return False
#         # The squares overlap!
#         return True
# a=[]
# compute(tubes,a)

# # compute(tubes)

In [61]:
# mylist = list(dict.fromkeys(a))
# print(mylist)

In [62]:
# f = open("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/data/overlap_frame.txt", "w")
# f.write(str(mylist))
# f.close()

From the relation map, we need to compute the Potential Collision Graph. This graph is composed by two types of nodes: 

- main-node (m-node): representing a tube
- sub-node (s-node): representing a potential collision between two tubes

The following is a summary of how the nodes are generated from the relations map: 

- We generate a m-node for each tube
- For each intersection between $T_1$ and $T_2$: 
    - we generate a s-node in $T_1$ containing the frame of intersection from $T_1$
    - we generate a s-node in $T_2$ containing the frame of intersection from $T_2$
- For each overlapping between $T_1$ and $T_2$:
    - we generate two s-node in $T_1$, one for the starting frame and one for the ending frame
    - we generate two s-node in $T_2$, one for the starting frame and one for the ending frame

After the node generation step, we need to wire the nodes with edges. The process is summarized as follows: 

- For each intersection: 
    - add an edge weighted 1 between the two s-nodes
- For each overlapping: 
    - add a directed edge between the starting and ending node weighted using $\Delta t$
    - add an edge between the starting nodes of $T_1$ and $T_2$ weighted -1
    - add an edge between the ending nodes of $T_1$ and $T_2$ weighted -1
    
> With $\Delta t$ we refer to the difference in frames between the end and the start of the collision

In [63]:
from pcgvs.aggregation.graph import PCG

pcg = PCG(tubes, relations)

We use the graph coloring technique from He Et al. paper, based on a mix between the DSATUR algorithm and the RFL algorithm, adapted for the L(q)-coloring problem. 

A brief description of the algorithm is presented here: 

1. Set the initial color $c=1$
2. Let $U$ be the list of the uncolored nodes
3. Compute the saturation values (see eq. (2) from He Et al.)
3. Order $U$ by saturation values
4. For each node $v \in U$ check if the coloring conditions are met (see Cond.1 and Cond.2 from He Et al.)
    1. If the conditions are met, color the node with $c$
    2. If the node is generated from an overlapping, color the directly connected node with $c \pm \Delta t$ (the sign is determined by the direction) 
5. Increase $c = c+1$ and repeat the process from step 2. until $U = \emptyset$

Check the saturation computation at `pcgvs.aggregation.coloring.SaturationCache`.

A suggested q value (q=3) for cars is proposed by the authors, so we set the parameter here:

In [64]:
# q=3

In [65]:
from pcgvs.aggregation.coloring import color_graph

color_graph(pcg, q)

100%|██████████| 944/944 [00:05<00:00, 177.67it/s] 


Then we use the *Condensation optimization* from Allegra Et al. in order to calculate the tubes starting time in the video synopsis: 

In [66]:
from pcgvs.aggregation.coloring import tubes_starting_time

starting_times = tubes_starting_time(pcg, q)
starting_times

{1: 1,
 4: 9,
 6: 17,
 8: 25,
 10: 33,
 11: 41,
 14: 49,
 17: 57,
 19: 65,
 20: 73,
 21: 81,
 23: 89,
 24: 97,
 25: 105,
 26: 113,
 27: 121,
 28: 129,
 30: 137,
 31: 145,
 32: 153,
 34: 161,
 35: 169,
 36: 177,
 38: 185}

Later we will use starting times to generate the synopsis. 

In [67]:
import json

with open('./synopsis/starting_times.json', 'w') as ssfile:
    writablejson = { str(k):v for k, v in starting_times.items() }
    json.dump(writablejson, ssfile)

In [46]:

# df= load_tubes_with_pandas(p)
# # path="/Users/nguyenduy/Desktop/pcgvs-main/notebooks/synopsis/patches"
# # images = [f for f in os.listdir(path) if os.path.splitext(f)[-1] == '.jpg']
# # for i in images:
# bg = cv2.imread("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/synopsis/patches/1_169.jpg")
# im= cv2.imread("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/synopsis/background.jpg")

# # Create an all white mask
# mask = 255 * np.ones(bg.shape, bg.dtype)

# # The location of the center of the src in the dst
# width, height, channels = im.shape
# center = (height//2, width//2)


# # Seamlessly clone src into dst and put the results in output
# normal_clone = cv2.seamlessClone(bg[df['x'][0],df['y'][0]], im, mask, center, cv2.NORMAL_CLONE)
# # mixed_clone = cv2.seamlessClone(bg, im, mask, center, cv2.MIXED_CLONE)

# # Write results
# cv2.imwrite("/Users/nguyenduy/Desktop/pcgvs-main/notebooks/data/1.jpg", normal_clone)